## Assignment 1 - Named Entity Recognition using CRF model
## 01.CRF-Training Program
## Group 8
### Anurag Maharshi - 2023MT12125
### SRIRAM ROKKAM - 2023MT12251
### GODLAVEETI ANIL GOVIND - 2023MT12272
### KORUKONDA SOWMYA - 2023MT12203
### VIGNESHWARAN K R - 2023mt12091

#### Importing Libraries

In [ ]:
! pip install sklearn-crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import io
import requests

In [ ]:
# Replace with your Google Drive file ID
file_id = "1c_Plf_fqKDjrzAj-GurD4rlmpp0yT2-R"

# Construct the download URL
download_url = f"https://docs.google.com/uc?export=download&id={file_id}"

# Download the file content
response = requests.get(download_url)
response.raise_for_status()  # Raise an exception for bad responses

# Read the CSV data into a pandas DataFrame
df = pd.read_csv(io.StringIO(response.text))



In [ ]:
#Display first 10 rows
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [ ]:
df.describe()

,Sentence #,Word,POS,Tag
count,47959,1048565,1048575,1048575
unique,47959,35177,42,17
top,Sentence: 1,the,NN,O
freq,1,52573,145807,887908


#### Observations :
* Unique words in the dataset = 35177.
* Unique tags = 17

In [ ]:
#Displaying the unique Tags
df['Tag'].unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [ ]:
#Checking null values, if any.
df.isnull().sum()

,0
Sentence #,1000616
Word,10
POS,0
Tag,0


Filling missing values


In [ ]:
df = df.fillna(method = 'ffill')

<ipython-input-8-569251c1683b>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method = 'ffill')


In [ ]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]

    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [ ]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

<ipython-input-9-a90e6ce4836c>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.grouped = self.df.groupby("Sentence #").apply(agg)


'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [ ]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


Getting all the sentences in the dataset.

In [ ]:
sentences = getter.sentences

#### Feature Preparation


In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [ ]:
## Splitting in train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
## defining CRF and training
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    max_iterations=100)

In [ ]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

#### Evaluating the model performance.
Using precision, recall and f1-score metrics to evaluate the performance of the model.

In [ ]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.9713891568201783


In [ ]:
report = flat_classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

       B-art       0.61      0.23      0.33        84
       B-eve       0.44      0.34      0.38        56
       B-geo       0.86      0.91      0.88      7452
       B-gpe       0.97      0.95      0.96      3075
       B-nat       0.65      0.38      0.48        45
       B-org       0.79      0.74      0.76      3976
       B-per       0.86      0.83      0.84      3408
       B-tim       0.91      0.88      0.90      4090
       I-art       0.38      0.05      0.10        55
       I-eve       0.35      0.19      0.25        58
       I-geo       0.82      0.80      0.81      1403
       I-gpe       0.89      0.74      0.81        43
       I-nat       0.43      0.23      0.30        13
       I-org       0.80      0.81      0.80      3320
       I-per       0.86      0.89      0.87      3498
       I-tim       0.80      0.76      0.78      1338
           O       0.99      0.99      0.99    179126

    accuracy              

In [ ]:
# Correctly importing metrics from sklearn_crfsuite
from sklearn_crfsuite import metrics


# Calculate and print metrics
print("Validation Metrics:")
print(f"Accuracy: {metrics.flat_accuracy_score(y_test, y_pred):.2f}")
print(f"Precision: {metrics.flat_precision_score(y_test, y_pred, average='weighted'):.2f}")
print(f"Recall: {metrics.flat_recall_score(y_test, y_pred, average='weighted'):.2f}")

print(f"F1 Score: {metrics.flat_f1_score(y_test, y_pred ,average='weighted'):.2f}")


Validation Metrics:
Accuracy: 0.97
Precision: 0.97
Recall: 0.97
F1 Score: 0.97


In [ ]:
## installing spacy to get POS from sentence. This is required for input in model
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
## this method will predict the tags of sentence

def predict_sentence(crf_model, sentence):
    # Assume sentence is a list of tuples [(word, pos), ...]
    # Extract the features from the sentence
    features = sent2features(sentence)

    # Predict the tags
    predicted_tags = crf_model.predict_single(features)

    return predicted_tags



In [ ]:
## function for splotting sentence and finding pos for each word
def sentence_to_word_pos(sentence):
    # Process the sentence using spaCy
    doc = nlp(sentence)

    # Extract words and their POS tags
    word_pos = [(token.text, token.tag_) for token in doc]

    return word_pos


In [ ]:
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Test it with an example sentence
doc = nlp("Thousands of people protested in New York.")
for token in doc:
    print(f'{token.text} -> {token.pos_}')

Thousands -> NOUN
of -> ADP
people -> NOUN
protested -> VERB
in -> ADP
New -> PROPN
York -> PROPN
. -> PUNCT


In [ ]:
plain_sentence = "I like staying in India rather Canada."

# Get the (word, POS) pairs
word_pos_pairs = sentence_to_word_pos(plain_sentence)

# Print the result
print(word_pos_pairs)

[('I', 'PRP'), ('like', 'VBP'), ('staying', 'VBG'), ('in', 'IN'), ('India', 'NNP'), ('rather', 'RB'), ('Canada', 'NNP'), ('.', '.')]


In [ ]:
plain_sentence = "Apple is iphone manufacturer and has office in India rather Canada."

# Get the (word, POS) pairs
word_pos_pairs2 = sentence_to_word_pos(plain_sentence)

predicted_tags = predict_sentence(crf, word_pos_pairs2)
print(predicted_tags)


# Print the results
for (word, pos), tag in zip(word_pos_pairs2, predicted_tags):
    print(f"Word: {word}, POS: {pos}, Predicted NER Tag: {tag}")

['B-org', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'B-geo', 'O']
Word: Apple, POS: NNP, Predicted NER Tag: B-org
Word: is, POS: VBZ, Predicted NER Tag: O
Word: iphone, POS: NN, Predicted NER Tag: O
Word: manufacturer, POS: NN, Predicted NER Tag: O
Word: and, POS: CC, Predicted NER Tag: O
Word: has, POS: VBZ, Predicted NER Tag: O
Word: office, POS: NN, Predicted NER Tag: O
Word: in, POS: IN, Predicted NER Tag: O
Word: India, POS: NNP, Predicted NER Tag: B-geo
Word: rather, POS: RB, Predicted NER Tag: O
Word: Canada, POS: NNP, Predicted NER Tag: B-geo
Word: ., POS: ., Predicted NER Tag: O


In [ ]:
## Exporting the model in google drive for furthur use
import joblib

# Assuming crf is your trained CRF model
# Save the model to a file (e.g., crf_model.pkl)



In [ ]:
# ## un comment below to save in google drive
# from google.colab import drive
# drive.mount('/content/drive')

# ## google drive path to store model
# dir='drive/MyDrive/Colab Notebooks/nlp_dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## saving in google drive for future -- uncomment below line also
# joblib.dump(crf, dir+'crf_model3.pkl')

In [ ]:
# saving in google colab session. It will be removed once session is removed

joblib.dump(crf, 'crf_model3.pkl')


['crf_model3.pkl']

In [ ]:
## using a new varible to load from memory and test out put
crf2a = joblib.load('crf_model3.pkl')

# Now you can use the loaded model for prediction
# For example, to predict NER tags for new sentences
new_sentence_features = sentence_to_word_pos("Thousands of people protested in India.")


predicted_tags = predict_sentence(crf2a, new_sentence_features)
print(predicted_tags)


# Print the results
for (word, pos), tag in zip(new_sentence_features, predicted_tags):
    print(f"Word: {word}, POS: {pos}, Predicted NER Tag: {tag}")

['O', 'O', 'O', 'O', 'O', 'B-geo', 'O']
Word: Thousands, POS: NNS, Predicted NER Tag: O
Word: of, POS: IN, Predicted NER Tag: O
Word: people, POS: NNS, Predicted NER Tag: O
Word: protested, POS: VBN, Predicted NER Tag: O
Word: in, POS: IN, Predicted NER Tag: O
Word: India, POS: NNP, Predicted NER Tag: B-geo
Word: ., POS: ., Predicted NER Tag: O
